In [0]:
import os
import pandas as pd 
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from keras.applications.resnet50 import ResNet50
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import EarlyStopping
from keras.models import Model, Sequential
from keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.utils import layer_utils
from sklearn.model_selection import train_test_split

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
import os
os.listdir('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/train')[:2]

['e7ab11f849e895ced3cdddf4d90e5bd2.jpg',
 'e7ab74acdc860dfb93271901da085ffd.jpg']

In [0]:
labels_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/labels.csv')


In [6]:
labels_df.head(2)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo


In [7]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
id       10222 non-null object
breed    10222 non-null object
dtypes: object(2)
memory usage: 159.8+ KB


In [0]:
def dog_breed_from_id(dog_id):
    str=""
    label=labels_df[labels_df['id'] ==dog_id]['breed'].values
    for i in label[:1]:
      str+=i
    return str

In [9]:
dog_breed_from_id('24bad9c547fc56016625af535b1070c1')

'borzoi'

In [15]:
#for i,img in tqdm(enumerate(os.listdir('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/train'))):
  #print(img)
  id,ext=os.path.splitext(img)
  breed=dog_breed_from_id(id)
  #print(id)
  if i<=7000:
    img1 = cv2.imread('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/train/'+img)
    if img1.any():
     #cv2.imshow('image',img1)
      if  not os.path.exists('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/training_1/'+breed):
        os.makedirs('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/training_1/'+breed)

        cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/training_1/'+breed+'/'+id+ext,img1)
      else:
        cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/training_1/'+breed+'/'+id+ext,img1)
  else:
    img1 = cv2.imread('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/train/'+img)
    if img1.any():
     #cv2.imshow('image',img1)
      if  not os.path.exists('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/val_1/'+breed):
        os.makedirs('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/val_1/'+breed)

        cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/val_1/'+breed+'/'+id+ext,img1)
      else:
        cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/val_1/'+breed+'/'+id+ext,img1)  

  


0it [00:00, ?it/s]
4it [00:00, 36.67it/s]
9it [00:00, 39.28it/s]
14it [00:00, 41.01it/s]
19it [00:00, 42.07it/s]
25it [00:00, 44.85it/s]
30it [00:00, 45.92it/s]
35it [00:00, 44.74it/s]
41it [00:00, 46.94it/s]
46it [00:00, 47.72it/s]
51it [00:01, 47.76it/s]
57it [00:01, 47.89it/s]
62it [00:01, 46.94it/s]
67it [00:01, 46.75it/s]
73it [00:01, 48.02it/s]
78it [00:01, 48.10it/s]
83it [00:01, 47.60it/s]
89it [00:01, 49.66it/s]
94it [00:01, 48.88it/s]
99it [00:02, 47.12it/s]
104it [00:02, 45.21it/s]
110it [00:02, 46.85it/s]
115it [00:02, 47.62it/s]
121it [00:02, 48.99it/s]
126it [00:02, 48.06it/s]
132it [00:02, 49.68it/s]
138it [00:02, 51.03it/s]
144it [00:02, 51.55it/s]
150it [00:03, 51.26it/s]
156it [00:03, 44.39it/s]
161it [00:03, 45.68it/s]
168it [00:03, 49.24it/s]
174it [00:03, 51.52it/s]
180it [00:03, 52.73it/s]
186it [00:03, 53.54it/s]
192it [00:03, 53.76it/s]
198it [00:04, 54.09it/s]
204it [00:04, 49.47it/s]
210it [00:04, 49.57it/s]
216it [00:04, 49.77it/s]
222it [00:04, 51.29it/s]
2

In [21]:
#General idea: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
#The author of this article tried to classify dogs vs cats with a pretrained model and provides an abstract idea 
model = ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [22]:
x = model.output
x = Flatten()(x)
x=Dropout(0.35)(x)
x=Dense(units=1000,activation='relu')(x)
x=Dropout(0.4)(x)
x=Dense(units=750,activation='relu')(x)
x=Dropout(0.4)(x)
x=Dense(units=1000,activation='relu')(x)
x=Dropout(0.8)(x)
#clf.add(Dense(units=120,activation='softmax')
#stochastic gradient descent -Adam -optimizer
#loss func categorical cross entropy
#metrics = accuracy
#clf.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
predictions = Dense(120, activation='softmax')(x)
main_model = Model(inputs=model.input, outputs=predictions)
#train only the hidden layers and output layer, donot train the resnet model
for curLayer in model.layers:
    curLayer.trainable = False
    
main_model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [23]:
main_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [24]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
       '/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/training_1/',
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
       '/content/gdrive/My Drive/Colab Notebooks/Data_sets/dog_breed/val_1/',
        target_size=(224, 224),
        batch_size=22,
        class_mode='categorical')

Found 6991 images belonging to 120 classes.
Found 3231 images belonging to 120 classes.


In [0]:
#training_set.class_indices

In [0]:
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]

main_model.fit_generator(
        training_set,
        steps_per_epoch=40,
        epochs=2,
        validation_data=test_set,
        validation_steps=101,callbacks=callbacks_list
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
 2/40 [>.............................] - ETA: 7:06 - loss: 13.6804 - acc: 0.0000e+00 